# Parcial de Regresión Lineal


## Preguntas de Negocio
1. **Predicción:** Si llega un carro con 92670.5 km, ¿a cuánto debería venderlo según el modelo?
2. **Inversión:** Como inversionista, ¿cuáles son los vehículos (según su kilometraje) que podrían generar un margen de ganancia atractivo?


In [46]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

%matplotlib inline

## 1. Cargar y Preprocesar Datos

Se descarga el dataset de vehículos, se descomprime y se carga en un DataFrame. Además, se eliminan los valores faltantes.

In [47]:
# Descargar y descomprimir el dataset
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/cars_dataset.zip
!unzip -o cars_dataset.zip


--2025-03-12 21:00:36--  https://github.com/javierherrera1996/lecture_analytics/raw/main/cars_dataset.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/cars_dataset.zip [following]
--2025-03-12 21:00:36--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/cars_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19753181 (19M) [application/zip]
Saving to: ‘cars_dataset.zip.1’

cars_dataset.zip.1  100%[===================>]  18.84M  --.-KB/s    in 0.06s   

2025-03-12 21:00:36 (323 MB/s) - ‘cars_dataset.zip.1’ saved [19753

In [48]:
pd.read_csv('car_prices.csv')
df=pd.read_csv('car_prices.csv')
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [49]:
df.isnull().sum()

,0
year,0
make,10301
model,10399
trim,10651
body,13195
transmission,65352
vin,4
state,0
condition,11820
odometer,94


## 2. Análisis Exploratorio de Datos (EDA)

Se generan estadísticas descriptivas para analizar las variables numéricas y entender la distribución de los datos.

In [50]:
df.describe()

,year,condition,odometer,mmr,sellingprice
count,558837.000000,547017.000000,558743.000000,558799.000000,558825.000000
mean,2010.038927,30.672365,68320.017767,13769.377495,13611.358810
std,3.966864,13.402832,53398.542821,9679.967174,9749.501628
min,1982.000000,1.000000,1.000000,25.000000,1.000000
25%,2007.000000,23.000000,28371.000000,7100.000000,6900.000000
50%,2012.000000,35.000000,52254.000000,12250.000000,12100.000000
75%,2013.000000,42.000000,99109.000000,18300.000000,18200.000000
max,2015.000000,49.000000,999999.000000,182000.000000,230000.000000


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558799 non-null  float64
 14  sellingprice  558825 non-null  float64
 15  saledate      558825 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 68.2+ MB


## 3. Modelo de Regresión Lineal

Se utiliza la variable 'odometer' para predecir 'sellingprice'. Se dividen los datos en conjunto de entrenamiento y prueba (80%-20%).

In [63]:
X = df[["odometer"]]
y = df["sellingprice"]

In [64]:
X.fillna(X.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)

<ipython-input-64-dcfa16c93e41>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)


In [65]:
X_train, X_test, y_train, y_test = train_test_split(
                                                  X,  # Asegurar 2D para sklearn
                                                  y,
                                                  test_size=0.2,
                                                  random_state=42
                                              )

print(f"Entrenamiento: {X_train.shape[0]} muestras")
print(f"Prueba: {X_test.shape[0]} muestras")

Entrenamiento: 447069 muestras
Prueba: 111768 muestras


In [66]:
model = LinearRegression()

In [67]:
model.fit(X_train, y_train)

LinearRegression()

In [69]:
intercept = model.intercept_
coef = model.coef_[0]

print(f"Intercept (beta_0): {intercept:.2f}")
print(f"Coeficiente (beta_1): {coef}")

Intercept (beta_0): 20860.13
Coeficiente (beta_1): -0.1060274463931155


## 4. Evaluación del Modelo

Se evalúa el desempeño del modelo en el conjunto de prueba utilizando métricas como MSE, MAE y R².

In [70]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [71]:
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mae_train= mean_absolute_error(y_train, y_pred_train)

In [72]:
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print("Conjunto de Entrenamiento:")
print(f"MSE: {mse_train}")
print(f"R^2: {r2_train}")
print(f"MAE: {mae_train}")
print("\nConjunto de Prueba:")
print(f"MSE: {mse_test}")
print(f"R^2: {r2_test}")
print(f"MAE: {mae_test}")

Conjunto de Entrenamiento:
MSE: 63225314.017707594
R^2: 0.33717791129301444
MAE: 5496.867602724755

Conjunto de Prueba:
MSE: 61195294.400367975
R^2: 0.3469042956275835
MAE: 5461.051319817094


## 5. Predicción y Respuestas a Preguntas de Negocio

### Pregunta 1: Precio Estimado para un Carro con 92670.5 km

Utilizamos el modelo para predecir el precio de venta de un carro con 92670.5 km.

In [74]:
# Predicción para un carro con 92670.5 km
nuevo_odometro = np.array([[92670.5]])
precio_estimado = model.predict(nuevo_odometro)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [75]:
#Prediccion el Modelo
print(f"Precio estimado para un carro con 92670.5 km: ${precio_estimado[0]:.2f}")

Precio estimado para un carro con 92670.5 km: $11034.51


## 6. Análisis Crítico de Resultados (10 Puntos Cerrados)

Responde a las siguientes preguntas de manera cerrada (Sí/No). Estas preguntas facilitan una calificación rápida:

1. ¿El coeficiente negativo (-0.107) indica que a mayor kilometraje, menor precio de venta?
**Respuesta:** *Sí*, ya que por cada unidad que aumente el kilometraje, el precio de venta disminuira 0.107 unidades.
2. ¿Un R² de 0.336 sugiere que el modelo explica más del 50% de la variabilidad en el precio de venta?
**Respuesta**: *No*, ya que un R² de 0.336 significa que el modelo explica el 33.6% de la variabilidad en el precio de venta y este es menor a 50%, por lo tanto el modelo no explica más del 50% de la variabilidad en el precio de venta.
3. ¿El valor elevado del MSE indica que existen errores importantes en las predicciones?
**Respuesta:**: *Sí*, ya que este calcula el promedio de los errores al cuadrado, es decir que si tenemos un valor alto de MSE la diferencia entre los valores predichos y los valores reales es grande en promedio.
4. ¿El MSE obtenido es suficientemente bajo para garantizar predicciones precisas en todos los casos?
**Respuesta:** *No*, el valor de MSE (61195294.400) no es lo suficientemente bajo para indicar predicciones precisas ya que es un valor alto.  
5. ¿La eliminación de filas con valores faltantes puede reducir la robustez del modelo?
**Respuesta**: *Sí*, ya que un modelo entrenado sólo con un conjunto de datos "perfectos" y limitado puede ser menos adaptable a variaciones en los datos nuevos.
6. ¿Incluir variables adicionales como año, condición y marca podría mejorar la capacidad predictiva del modelo?
**Respuesta:** *Sí*, ya que son variables que influyen en el precio del vehículo. Los vehículos más nuevos tienden a tener un valor más alto, sí el estado del vehículo es bueno o excelente su valor aumenta y si el vehículo es de una marca de gama alta (Por ejemplo, BMW) su precio va a ser alto. Por tanto, al agregar estas variables el modelo tiene más información para predecir de manera más precisa el precio de venta.
7. ¿La relación negativa entre kilometraje y precio es consistente con el comportamiento esperado en el mercado de vehículos?
**Respuesta**: *Sí*, ya que la relación negaiva entre kilometraje y precio es constitante con el comportamiento esperado en el mercado de estos. A mayor kilometraje, menor precio de venta por la depreciación y la menor demanda de carros con mayor uso.
8. ¿Dividir los datos en 80% entrenamiento y 20% prueba es una práctica recomendada para evaluar el modelo?
**Respuesta**: Sí, es una practica recomendada, además de altamente usada, para evaluar el modelo porque proporciona suficientes datos para el entrenamiento y, al mismo tiempo, para los datos de prueba. Igualmente, la proporción se puede ajustar según el tamaño del conjunto de datos y la complejidad del modelo.
